In [ ]:
import pickle
import numpy as np
import torch
from tqdm import tqdm
import numqi
import matplotlib.pyplot as plt

In [ ]:
train_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Random generate a d^-1-dimensional real vector in Euclidean space
def random_vector(d):
    x = np.random.normal(size=d*d-1)
    x = x / np.linalg.norm(x)
    return x

def save_data(xdata, ydata, filepath='data_boundary.pkl'):
    with open(filepath, 'wb') as fid:
        pickle.dump(dict(xdata=xdata, ydata=ydata), fid)

def load_data(filepath='data_boundary.pkl'):
    with open(filepath, 'rb') as fid:
        tmp0 = pickle.load(fid)
        ret = tmp0['xdata'], tmp0['ydata']
    return ret


In [ ]:
dimA = 2
dimB = 2
dim = dimA * dimB
num_sample_random_mixed = 2000000
num_pure_product = 20000
num_around_pure_product = 100
num_sample_random_pure_product = num_pure_product * num_around_pure_product
num_sample = num_sample_random_mixed + num_sample_random_pure_product
batch_size = 4096
num_workers = 2

In [ ]:
# random data
xdata = []
ydata = []
vec_list = [random_vector(dim) for _ in range(num_sample_random_mixed//2)]
for vec in tqdm(vec_list):
    rho = numqi.gellmann.gellmann_basis_to_dm(vec)
    beta_ppt_l, beta_ppt_u = numqi.entangle.get_ppt_boundary(rho,(dimA,dimB))
    xdata.append(vec)
    ydata.append(beta_ppt_u)
    xdata.append(-vec)
    ydata.append(-beta_ppt_l)
xdata = np.array(xdata)
ydata = np.array(ydata)
print(f'Mixed ppt state: min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {np.mean(ydata)}, variance: {np.var(ydata)}')
save_data(xdata, ydata, f'data_ppt_mixed_boundary_{dimA}_{dimB}_{num_sample_random_mixed}.pkl') 

In [ ]:
# random data around pure product states
xdata = []
ydata = []
pure_product_list = [numqi.random.rand_separable_dm(dimA,dimB,k=1,pure_term=True) for _ in range(num_pure_product)]
for rho in tqdm(pure_product_list):
    vec = numqi.gellmann.dm_to_gellmann_basis(rho)
    for _ in range(num_around_pure_product):
        perturbation = np.random.normal(size=dim*dim-1, scale=1e-2)
        new_vec = vec + perturbation
        new_vec = new_vec / np.linalg.norm(new_vec)
        new_rho = numqi.gellmann.gellmann_basis_to_dm(new_vec)
        _, beta_ppt_u = numqi.entangle.get_ppt_boundary(new_rho,(dimA,dimB))
        xdata.append(new_vec)
        ydata.append(beta_ppt_u)
xdata = np.array(xdata)
ydata = np.array(ydata)
print(f'Pure product state: min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {np.mean(ydata)}, variance: {np.var(ydata)}')
save_data(xdata, ydata, f'data_pure_product_boundary_{dimA}_{dimB}_{num_sample_random_pure_product}.pkl') 

In [ ]:
print(np.sqrt(1/(2*dim*(dim-1))))
print(np.sqrt((dim-1)/(2*dim)))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, dim, device='cpu'):
        super(Net, self).__init__()
        tmp0 = [dim**2-1, 128, 256, 256, 512, 512, 512, 256, 256, 128, 1]
        self.fc_list = torch.nn.ModuleList([torch.nn.Linear(tmp0[x], tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.a = torch.sqrt(torch.tensor(1/(2*dim*(dim-1)), device=device))
        self.b = torch.sqrt(torch.tensor((dim-1)/(2*dim), device=device))
        self.scale = torch.tensor(0.05, device=device)

    def forward(self, x):
        for ind0 in range(len(self.fc_list)):
            y = self.fc_list[ind0](x)
            y = self.bn_list[ind0](y)
            if ind0<(len(self.fc_list)-1):
                y = torch.nn.functional.leaky_relu(y)
            if x.shape[-1]==y.shape[-1]:
                x = y + x
            else:
                x = y
        x = (1 + self.scale) * (self.b - self.a) * torch.sigmoid(x) + self.a * (1 - self.scale)
        x = x[:,0] 
        return x

In [ ]:
xdata_mixed,ydata_mixed = load_data(filepath=f'data_ppt_mixed_boundary_{dimA}_{dimB}_{num_sample_random_mixed}.pkl')
xdata_pure,ydata_pure = load_data(filepath=f'data_pure_product_boundary_{dimA}_{dimB}_{num_sample_random_pure_product}.pkl')
xdata = np.concatenate([xdata_mixed, xdata_pure], axis=0)
ydata = np.concatenate([ydata_mixed, ydata_pure], axis=0)
tmp0 = torch.tensor(xdata, dtype=torch.float32)
tmp1 = torch.tensor(ydata, dtype=torch.float32)
dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tmp0, tmp1), batch_size=batch_size, shuffle=True)

In [ ]:
model = Net(dim,device=train_device).to(train_device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)
loss_history = []
model.train()
for epoch in range(5):
    with tqdm(dataloader) as pbar:
        for x, y in pbar:
            x, y = x.to(train_device), y.to(train_device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = torch.nn.functional.mse_loss(y_pred, y)
            loss.backward()
            optimizer.step()
            loss_history.append(loss.item())
            pbar.set_postfix(train_loss='{:.5}'.format(sum(loss_history[-10:])/10))
        lr_scheduler.step()

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(len(loss_history)), loss_history, '.')
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
# log scale
ax.set_yscale('log')
ax.grid()
fig.tight_layout()

In [ ]:
torch.save(model.state_dict(), f'model_ppt_{dimA}_{dimB}_{num_sample}_{batch_size}.pt')

In [ ]:
i=13
j=14
GellMann = numqi.gellmann.all_gellmann_matrix(dim, with_I=False)
op0 = GellMann[i] 
op1 = GellMann[j] 

theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

_, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_ppt_cs = np.array([numqi.entangle.get_ppt_boundary(hf_plane(x), (dimA,dimB))[1] for x in theta_list])
nr_ppt_cs = beta_ppt_cs.reshape(-1,1) * direction

beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_list = np.array([numqi.gellmann.dm_to_gellmann_basis(hf_plane(x)) for x in theta_list]).reshape(-1, dim**2-1)
model = Net(dim)
device = torch.device('cpu')
model.load_state_dict(torch.load(f'model_ppt_{dimA}_{dimB}_{num_sample}_{batch_size}.pt', map_location=device))
model.eval()
with torch.no_grad():
    beta_ppt_model = model(torch.tensor(vec_list, dtype=torch.float32)).numpy()

nr_ppt_model_cs = beta_ppt_model.reshape(-1,1) * direction

nr_dm_inside_ball = model.a.numpy() * direction
nr_dm_outside_ball = model.b.numpy() * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-', color='red')
ax.plot(nr_ppt_cs[:,0], nr_ppt_cs[:,1], '-', color='blue')
ax.plot(nr_ppt_model_cs[:,0], nr_ppt_model_cs[:,1], '--', color='blue')
ax.plot(nr_dm_inside_ball[:,0], nr_dm_inside_ball[:,1], '-', color='green')
ax.plot(nr_dm_outside_ball[:,0], nr_dm_outside_ball[:,1], '-', color='green')
ax.axis('equal')
ax.set_xlabel(r'$\lambda_{%d}$'%(i+1))
ax.set_ylabel(r'$\lambda_{%d}$'%(j+1))
#fig.savefig(f'ppt_{dimA}_{dimB}_{i+1}_{j+1}.pdf')

In [ ]:
op0 = numqi.random.rand_density_matrix(dim)
op1 = numqi.random.rand_density_matrix(dim)

theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

_, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_ppt_cs = np.array([numqi.entangle.get_ppt_boundary(hf_plane(x), (dimA,dimB))[1] for x in theta_list])
nr_ppt_cs = beta_ppt_cs.reshape(-1,1) * direction

beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_list = np.array([numqi.gellmann.dm_to_gellmann_basis(hf_plane(x)) for x in theta_list]).reshape(-1, dim**2-1)
model = Net(dim)
device = torch.device('cpu')
model.load_state_dict(torch.load(f'model_ppt_{dimA}_{dimB}_{num_sample}_{batch_size}.pt', map_location=device))
model.eval()
with torch.no_grad():
    beta_ppt_model = model(torch.tensor(vec_list, dtype=torch.float32)).numpy()

nr_ppt_model_cs = beta_ppt_model.reshape(-1,1) * direction

nr_dm_inside_ball = model.a.numpy() * direction
nr_dm_outside_ball = model.b.numpy() * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-', color='red')
ax.plot(nr_ppt_cs[:,0], nr_ppt_cs[:,1], '-', color='blue')
ax.plot(nr_ppt_model_cs[:,0], nr_ppt_model_cs[:,1], '--', color='blue')
ax.plot(nr_dm_inside_ball[:,0], nr_dm_inside_ball[:,1], '-', color='green')
ax.plot(nr_dm_outside_ball[:,0], nr_dm_outside_ball[:,1], '-', color='green')
ax.axis('equal')
#fig.savefig(f'ppt_{dimA}_{dimB}_random.pdf')



In [ ]:
op0 = numqi.state.Werner(dimA, 1)
op1 = numqi.state.Isotropic(dimA, 1)

theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

theta1, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_ppt_cs = np.array([numqi.entangle.get_ppt_boundary(hf_plane(x), (dimA,dimB))[1] for x in theta_list])
nr_ppt_cs = beta_ppt_cs.reshape(-1,1) * direction

beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_list = np.array([numqi.gellmann.dm_to_gellmann_basis(hf_plane(x)) for x in theta_list]).reshape(-1, dim**2-1)
model = Net(dim)
device = torch.device('cpu')
model.load_state_dict(torch.load(f'model_ppt_{dimA}_{dimB}_{num_sample}_{batch_size}.pt', map_location=device))
model.eval()
with torch.no_grad():
    beta_ppt_model = model(torch.tensor(vec_list, dtype=torch.float32)).numpy()

nr_ppt_model_cs = beta_ppt_model.reshape(-1,1) * direction

nr_dm_inside_ball = model.a.numpy() * direction
nr_dm_outside_ball = model.b.numpy() * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-', color='red')
ax.plot(nr_ppt_cs[:,0], nr_ppt_cs[:,1], '-', color='blue')
ax.plot(nr_ppt_model_cs[:,0], nr_ppt_model_cs[:,1], '--', color='blue')
ax.plot(nr_dm_inside_ball[:,0], nr_dm_inside_ball[:,1], '-', color='green')
ax.plot(nr_dm_outside_ball[:,0], nr_dm_outside_ball[:,1], '-', color='green')
label0 = 'werner'
label1 = 'isotropic'
for theta, label in [(0,label0),(theta1,label1)]:
    radius = model.b.numpy()
    ax.plot([0, radius*np.cos(theta)], [0, radius*np.sin(theta)], linestyle=':', label=label)
ax.axis('equal')
ax.legend()
#fig.savefig(f'ppt_{dimA}_{dimB}_{label0}_{label1}.pdf')
